# Analisis Exploratorio TP1

In [ ]:
import numpy as np 
import pandas as pd
import datetime
#import gmplot 

import seaborn as sns

from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

import matplotlib.pyplot as plt
%matplotlib inline

#  Analizamos las estaciones (station.csv)

In [ ]:
stations = pd.read_csv("../input/station.csv")
stations.head()

La cantidad de estaciones es:

In [ ]:
print (str(len(stations.id.unique())))
print (str((stations.city.unique())))

Los nombres de las 70 estaciones son:

In [ ]:
print (stations.name.unique()) 
print (stations.id.unique())

Cantidad de "docks" (espacios para estacionar las bicicletas):

In [ ]:
print (str(stations.dock_count.sum()))

¿Las estaciones fueron instaladas el mismo día?

In [ ]:
stations_installed = stations
stations_installed.installation_date = pd.to_datetime(stations_installed.installation_date, format='%m/%d/%Y') #Pase a formato datetime las fechas porque es mas comodo
stations_installed.plot(x='installation_date' , y = 'id', style='go',figsize=(14,5))

plt.show()

# Analizamos los viajes (trip.csv)

In [ ]:
trips = pd.read_csv("../input/trip.csv")
trips.start_date = pd.to_datetime(trips.start_date, format='%m/%d/%Y %H:%M') #Pase a formato datetime las fechas porque es mas comodo
trips.end_date = pd.to_datetime(trips.end_date, format='%m/%d/%Y %H:%M')
trips.head()

# Analizamos si figuran la misma cantidad de estaciones que en stations.csv

¿Las estaciones que figuran en stations.csv coinciden con las que se registraron en trip.csv? 

In [ ]:
print (str(len(trips.start_station_id.unique())))
print (str(len(trips.end_station_id.unique())))
print (str(len(trips.start_station_name.unique())))
print (str(len(trips.end_station_name.unique())))

print (str((trips.start_station_id.unique())))

Vemos que figuran 74 estaciones nombradas diferente pero seguimos contando 70 stations id.
Veamos si los 70 ids de ambos csv coinciden. Lo mismo para los nombres.

In [ ]:
# print (str(len(df.start_station_name.unique())))
stations_d = {}
for name in stations.name:
    stations_d[name] = 0;

stations_x = {}
for name in trips.start_station_name:
    if name not in stations_d:
        if name in stations_x:
            stations_x [name] += 1
        else:
            stations_x[name] = 0
stations_x = {}
for name in trips.end_station_name:
    if name not in stations_d:
        if name in stations_x:
            stations_x [name] += 1
        else:
            stations_x[name] = 0
        
        
print(stations_x)  

In [ ]:

print ([x for x in  list(set(trips.start_station_name.unique())) if x not in list(set(stations.name.unique()))])

print ([x for x in  list(set(trips.start_station_id.unique())) if x not in list(set(stations.id.unique()))])

print (trips.loc[trips['start_station_name'] == 'San Jose Government Center','start_station_id'].unique(), trips.loc[trips['start_station_name'] == 'Washington at Kearny','start_station_id'].unique(), trips.loc[trips['start_station_name'] == 'Post at Kearny','start_station_id'].unique(), trips.loc[trips['start_station_name'] == 'Broadway at Main','start_station_id'].unique())

Los ids presentes en ambos csv son los mismos, la diferencia esta en los nombres de las estaciones. Para las estaciones con id: 25, 46, 47, 80 los nombres no coinciden con la informacion registrada en stations.csv
Comparemos como se dan las diferencias:

In [ ]:
d={}
dok = {}

for id in stations.id.unique():   
    d[id] = list((set((trips.loc[trips['start_station_id'] == id, 'start_station_name']).tolist())))
    dok[id] = (stations.loc[stations['id'] == id, 'name']).to_string()

try:
    test =  pd.DataFrame(dok.items(), columns=['id','station_name'])
    test = test.set_index('id')
    test['trips_names'] = pd.Series(d, name='name')
except Exception:
    test =  pd.DataFrame()

test.head(70)

Conclusion:
    Los ids se respetan con su correspondiente estación.
Tenemos casos mal seteados:
        id: 25 corresponde a la estación: Stanford in Redwood City. En los registros de viajes es usado para      Stanford in Redwood City y Broadway at Main.
        id: 46 corresponde a la estación: Washington at Kearney. En los registros de viajes es usasdo para Washington at Kearney, Washington at Kearny. En este caso claramente el error es de tipeo.
        id: 47 corresponde a la estación: Post at Kearney En los registros de viajes es usasdo para Post at Kearney y Post at Kearny. Ocurre lo mismo que para el punto anterior
        id: 80 corresponde a la estación: Santa Clara County Civic Center. En los registros de viajes es usado para San Jose Government Center y Santa Clara County Civic Center

In [ ]:
print (trips.loc[trips['start_station_name'] == 'San Jose Government Center','start_station_id'].count())
print (trips.loc[trips['end_station_name'] == 'San Jose Government Center','end_station_id'].count())
print (trips.loc[trips['start_station_name'] == 'Washington at Kearny','start_station_id'].count())
print (trips.loc[trips['end_station_name'] == 'Washington at Kearny','end_station_id'].count())
print (trips.loc[trips['start_station_name'] == 'Post at Kearny','start_station_id'].count())
print (trips.loc[trips['end_station_name'] == 'Post at Kearny','end_station_id'].count())
print (trips.loc[trips['start_station_name'] == 'Broadway at Main','start_station_id'].count())
print (trips.loc[trips['end_station_name'] == 'Broadway at Main','end_station_id'].count())

Opciones:
    Dado al hecho que para el primer y último caso son muy pocos los registros podríamos a borrar todas aquellas filas cuyo station name sea uno de esos. Para los casos 2 y 3 se corregiría Kearny por Kearney
    Borrar todos los casos que presenten estos station names
    Dejar todos y no hacer nada

In [ ]:
print (str(len(trips.zip_code.unique())))

Las duraciones están en segundos, para el análisis conviene pasar a minuto.

In [ ]:
trips_in_minute = trips
trips_in_minute.duration /= 60
trips_in_minute.head()

Verificamos si hay datos nulos:

In [ ]:
trips_in_minute.isnull().sum()

Resumen estadístico del dataframe:

In [ ]:
trips_in_minute.describe()

Tenemos un mínimo de 1 minuto, lo que no va a ser considerado como viaje.
y un máximo de 287840 lo que corresponde a 199.9 días.

Por lo tanto vamos a graficar las duraciones por viaje para identificar valores outliers y luego recortar esos datos.

In [ ]:
trips_in_minute.plot(x='start_date' , y = 'duration', style='o',figsize=(15,10))

plt.show()

Analizando el gráfico y los valores que se vieron en el resumen estadístico vamos a recortar aquellos viajes cuya duración sea menor a 370 minutos y mayor a 3 minutos.

In [ ]:
trips_in_minute = trips_in_minute[trips_in_minute.duration <= 370]
trips_in_minute = trips_in_minute[trips_in_minute.duration >3]

trips_in_minute.plot(x='start_date' , y = 'duration', style='o',figsize=(20,10))

plt.show()

Podemos ver una zona muy oscura entre los viajes de duración entre 3 y 50 minutos.

Ahora vamos a analizar la cantidad de viajes contra los distintos días de la semana.

In [ ]:
trips_weekday = trips_in_minute
trips_weekday['weekday'] =  pd.to_datetime(trips_weekday['start_date']).dt.weekday_name
trips_weekday.groupby('weekday').count()['start_date'].sort_values(ascending=False)[0:19].plot(kind='bar',figsize=(14,4));

Vemos que los fines de semana se realiazaron notablemente menor cantidad de viajes.

Podemos ahora analizar la duración promedio de los viajes para cada dia de semana

In [ ]:
trips_weekday.groupby('weekday').mean()['duration'].sort_values(ascending=False)[0:7].plot(kind='bar',figsize=(14,4));

El último grafico nos muestra claramente que los fines de semana la duración de los viajes es mucho mayor que la de los días de semana.

Ahora analizamos la cantidad de viajes segun el mes: 

In [ ]:
trips_month = trips_in_minute
trips_month['month'] =  pd.to_datetime(trips_month['start_date']).dt.month

trips_month.groupby('month').count()['start_date'][0:12].plot(kind='bar',figsize=(13,5));

Ahora podríamos ver el clima según los meses para comprender mejor los resultados del gráfico anterior

# Analizamos el clima (weather.csv)

In [ ]:
weathers = pd.read_csv("../input/weather.csv")
weathers['month'] =  pd.to_datetime(weathers['date']).dt.month
weathers.head()

In [ ]:
weathers.groupby('month').mean()['mean_temperature_f'][0:12].plot(kind='line',figsize=(10,5));

Vemos que el promedio de temperaturas más altas se registra entre los meses de junio y octubre, lo que coincide con el incremento en la cantidad de viajes en esos meses, de todos modos no explicaría el pico de enero.

# Analizamos como influye las precipitaciones a la cantidad de viajes.

In [ ]:
# Convierto el campo date en datetime
weather_convert = weathers
weather_convert.date = pd.to_datetime(weather_convert.date)
weather_convert.precipitation_inches = pd.to_numeric(weather_convert.precipitation_inches, errors='coerce')
weather_convert = weather_convert[np.isfinite(weather_convert['precipitation_inches'])]
weather_convert.head()

In [ ]:
weather_convert.describe()

In [ ]:
weather_convert.info()

In [ ]:
# Compruebo que no se repitan las fechas
nUniqueDate = weather_convert.date.nunique()
print ("Fechas distintas:")
print (nUniqueDate)
nUniqueZipCode = weather_convert.zip_code.nunique()
print ("Zip codes distintos:")
print (nUniqueZipCode)

print (nUniqueDate*nUniqueZipCode)

Hay 733 fechas distintas por cada zip_code, como hay 5 zip_codes distintos => hay 3665 registros diferentes => no se repiten

In [ ]:
print(weather_convert.groupby('precipitation_inches').size())

# Analizamos cantidad de precipitación total por mes.

In [ ]:
weather_convert['month'] =  pd.to_datetime(weather_convert['date']).dt.month

grouped_weather = weather_convert.groupby('month')['precipitation_inches'].sum()

grouped_weather.plot(title="Suma de las precipitaciones totales por mes \n(acumulado desde el 29/8/2013 hasta el 31/8/2015)", xticks=[1,2,3,4,5,6,7,8,9,10,11,12])
plt.xlabel("mes")
plt.ylabel("precipitaciones [pulgadas]")

Según se puede ver en el grafico, los picos donde se dan mayores precipitaciones son en febrero y en diciembre, donde también se dan la menor cantidad de viajes.

También se da un pico menor en septiembre, donde se había notado una pequeña reducción en la cantidad de viajes.

In [ ]:
print (weather_convert.events.unique())
print (weather_convert.zip_code.unique())

In [ ]:
# Se pasa mean_temperature a grados celsius
weather_convert['mean_temperature_f'] = (weather_convert.mean_temperature_f - 32)/1.8

Analizamos el campo zip_code de la tabla 'weather' donde hay 5 claves distintas, y de 'trip' donde hay muchas claves que no corresponden a la bahia de san francisco.  Por lo tanto, creamos una tabla auxilar con los 5 zip_code correspondiente a dicha bahia.

In [ ]:
cityZipCodes = pd.DataFrame(data=[['San Francisco',94107],['San Jose',95113],['Redwood City',94063],['Mountain View',94041],['Palo Alto',94301]], columns=['city', 'zip_code'])
print (cityZipCodes)

In [ ]:
trip_convert = trips

trip_convert.zip_code = pd.to_numeric(trip_convert.zip_code, errors='coerce')
trip_convert = trip_convert[np.isfinite(trip_convert['zip_code'])]

trip_convert['dia'] = trip_convert.start_date.dt.weekday_name

trip_convert['start_date'] = trip_convert.start_date.dt.normalize()

In [ ]:
station_filter = stations.loc[:,['id','city']]
station_filter

In [ ]:
dfMergedZipCodes = pd.merge(weather_convert, cityZipCodes, how='inner', on='zip_code')
dfMergedZipCodes

In [ ]:
tripsMergedCity = pd.merge(trip_convert, station_filter, how='inner', left_on='start_station_id', right_on='id')
tripsMergedCity

In [ ]:
tripsMonday = tripsMergedCity[tripsMergedCity.dia == 'Monday']
tripsTuesday = tripsMergedCity[tripsMergedCity.dia == 'Tuesday']
tripsWednesday = tripsMergedCity[tripsMergedCity.dia == 'Wednesday']
tripsThursday = tripsMergedCity[tripsMergedCity.dia == 'Thursday']
tripsFriday = tripsMergedCity[tripsMergedCity.dia == 'Friday']
tripsSaturday = tripsMergedCity[tripsMergedCity.dia == 'Saturday']
tripsSunday = tripsMergedCity[tripsMergedCity.dia == 'Sunday']

In [ ]:
print (trip_convert.info())

In [ ]:
dfMergedMonday = pd.merge(tripsMonday, dfMergedZipCodes, left_on=['city','start_date'], right_on=['city','date'])
dfMergedThursday = pd.merge(tripsThursday, dfMergedZipCodes, left_on=['city','start_date'], right_on=['city','date'])
dfMergedWednesday = pd.merge(tripsWednesday, dfMergedZipCodes, left_on=['city','start_date'], right_on=['city','date'])
dfMergedTuesday = pd.merge(tripsTuesday, dfMergedZipCodes, left_on=['city','start_date'], right_on=['city','date'])
dfMergedFriday = pd.merge(tripsFriday, dfMergedZipCodes, left_on=['city','start_date'], right_on=['city','date'])
dfMergedSaturday = pd.merge(tripsSaturday, dfMergedZipCodes, left_on=['city','start_date'], right_on=['city','date'])
dfMergedSunday = pd.merge(tripsSunday, dfMergedZipCodes, left_on=['city','start_date'], right_on=['city','date'])

In [ ]:
dfMergedMonday = pd.merge(tripsMonday, dfMergedZipCodes, left_on=['city','start_date'], right_on=['city','date'])
dfMergedThursday = pd.merge(tripsThursday, dfMergedZipCodes, left_on=['city','start_date'], right_on=['city','date'])
dfMergedWednesday = pd.merge(tripsWednesday, dfMergedZipCodes, left_on=['city','start_date'], right_on=['city','date'])
dfMergedTuesday = pd.merge(tripsTuesday, dfMergedZipCodes, left_on=['city','start_date'], right_on=['city','date'])
dfMergedFriday = pd.merge(tripsFriday, dfMergedZipCodes, left_on=['city','start_date'], right_on=['city','date'])
dfMergedSaturday = pd.merge(tripsSaturday, dfMergedZipCodes, left_on=['city','start_date'], right_on=['city','date'])
dfMergedSunday = pd.merge(tripsSunday, dfMergedZipCodes, left_on=['city','start_date'], right_on=['city','date'])

In [ ]:
dfMergedGroupedCountMonday = dfMergedMonday.groupby('mean_temperature_f', as_index=False)['duration'].count()
dfMergedGroupedCountTuesday = dfMergedTuesday.groupby('mean_temperature_f', as_index=False)['duration'].count()
dfMergedGroupedCountWednesday = dfMergedWednesday.groupby('mean_temperature_f', as_index=False)['duration'].count()
dfMergedGroupedCountThursday = dfMergedThursday.groupby('mean_temperature_f', as_index=False)['duration'].count()
dfMergedGroupedCountFriday = dfMergedFriday.groupby('mean_temperature_f', as_index=False)['duration'].count()
dfMergedGroupedCountSaturday = dfMergedSaturday.groupby('mean_temperature_f', as_index=False)['duration'].count()
dfMergedGroupedCountSunday = dfMergedSunday.groupby('mean_temperature_f', as_index=False)['duration'].count()

In [ ]:
dfMergedGroupedCountMonday

Se agrupo por temperatura (el promedio del dia) y me quedo solo con las temperaturas en las cuales hay mas de 2000 viajes.
Me quedo con las temp. que estan entre 10.000000 <= T <= 22.222222 que son las que tienen almenos 2000 viajes

In [ ]:
dfMergedGroupedMeanMonday = dfMergedMonday.groupby('mean_temperature_f', as_index=False)['duration'].mean()
dfMergedGroupedMeanMonday.columns = ['mean_temperature_f', 'Lunes']
dfMergedGroupedMeanTuesday = dfMergedTuesday.groupby('mean_temperature_f', as_index=False)['duration'].mean()
dfMergedGroupedMeanTuesday.columns = ['mean_temperature_f', 'Martes']
dfMergedGroupedMeanWednesday = dfMergedWednesday.groupby('mean_temperature_f', as_index=False)['duration'].mean()
dfMergedGroupedMeanWednesday.columns = ['mean_temperature_f', 'Miercoles']
dfMergedGroupedMeanThursday = dfMergedThursday.groupby('mean_temperature_f', as_index=False)['duration'].mean()
dfMergedGroupedMeanThursday.columns = ['mean_temperature_f', 'Jueves']
dfMergedGroupedMeanFriday = dfMergedFriday.groupby('mean_temperature_f', as_index=False)['duration'].mean()
dfMergedGroupedMeanFriday.columns = ['mean_temperature_f', 'Viernes']
dfMergedGroupedMeanSaturday = dfMergedSaturday.groupby('mean_temperature_f', as_index=False)['duration'].mean()
dfMergedGroupedMeanSaturday.columns = ['mean_temperature_f', 'Sabado']
dfMergedGroupedMeanSunday = dfMergedSunday.groupby('mean_temperature_f', as_index=False)['duration'].mean()
dfMergedGroupedMeanSunday.columns = ['mean_temperature_f', 'Domingo']
#dfMergedGroupedMean

dfMergedGroupedMean = pd.merge(dfMergedGroupedMeanMonday, dfMergedGroupedMeanTuesday, on='mean_temperature_f')
dfMergedGroupedMean = pd.merge(dfMergedGroupedMean, dfMergedGroupedMeanWednesday, on='mean_temperature_f')
dfMergedGroupedMean = pd.merge(dfMergedGroupedMean, dfMergedGroupedMeanThursday, on='mean_temperature_f')
dfMergedGroupedMean = pd.merge(dfMergedGroupedMean, dfMergedGroupedMeanFriday, on='mean_temperature_f')
dfMergedGroupedMean = pd.merge(dfMergedGroupedMean, dfMergedGroupedMeanSaturday, on='mean_temperature_f')
dfMergedGroupedMean = pd.merge(dfMergedGroupedMean, dfMergedGroupedMeanSunday, on='mean_temperature_f')
dfMergedGroupedMean

In [ ]:
dfPlot = dfMergedGroupedMean[(dfMergedGroupedMean.mean_temperature_f >= 11.66666) & (dfMergedGroupedMean.mean_temperature_f <= 21.2)]
#plot(x='mean_temperature_f',y='duration')


dfPlot.plot(x='mean_temperature_f', title="Promedio de la duracion de los viajes segun temperatura media del dia",
            color=['#3336FF','#33FF49','#FF3333','#F3FF33','#55F1F1','#000000','#93881A'])
plt.legend = [1,2,3,4,5]
plt.xlabel("Temperatura [grad. celsius]")
plt.ylabel("Duracion [seg]")
#plt.ylim([0,9000])
#plt.colors = ['b','o','r','y','m']
#plt.figure()
#with pd.plot_params.use('x_compat', True):
 #   dfPlot.duration.plot(color='r')

In [ ]:
dfPlot

In [ ]:
dfPlot.info()

# Analizamos la duracion de cada estacion y la relacion entre si

In [ ]:
grupo_trip = trips_in_minute.loc[:,['start_station_id','duration']]

grupo_trip['subindex'] = grupo_trip.groupby('start_station_id').cumcount() + 1
grupo_trip = pd.pivot_table(grupo_trip,index='subindex',columns='start_station_id',values='duration')

grupo_trip.describe()

In [ ]:
grupo_trip.plot.box(figsize=(15,5));

In [ ]:
trips_in_minute.plot.scatter('end_station_id','start_station_id',alpha=1,title = "Relacion entre estaciones",figsize=(15,5));

In [ ]:
viajesInterCiudad = trips_in_minute[trips_in_minute.end_station_id == 2]
viajesInterCiudad = viajesInterCiudad[viajesInterCiudad.start_station_id > 60]
viajesInterCiudad.head()

 Cuál es la estación de donde salieron más viajes ?

In [ ]:
plt.figure()
plt.ylabel = "1"
trips_in_minute.groupby('start_station_name').count()['id'].sort_values(ascending=False)[0:30].plot(title = "Estaciones de donde salen mas viajes",kind='bar',figsize=(14,4))

Cuál es la estación a la que llegaron más viajes?

In [ ]:
trips_in_minute.groupby('end_station_name').count()['id'].sort_values(ascending=False)[0:30].plot(kind='bar',figsize=(14,5))

Se puede observar de los graficos, que la estacion de donde salieron y llegaron mas viajes es la estacion id: 70 (San Francisco Caltrain). Es la estacion con mas trafico.

# Concetración de estaciones con heatmap

In [ ]:
'''gmap = gmplot.GoogleMapPlotter(37.336, -121.894074, 16)
gmap.heatmap(stations.lat, stations.long)  
gmap.draw("concetraciónDeEstaciones.html")'''

En el mapa se puede ver dos puntos de mayor concentracion de estaciones, En san josé y en San francisco. Siendo ésta última la que mayor concentración de estaciones de bicicletas posee.

# Heatmap de donde salen más cantidad de viajes.

In [ ]:
'''googlePlotMap= trips_in_minute.loc[:,['start_station_id','end_station_id']]
googlePlotMap.columns = ['id' , 'end_station_id']
stationPlot = stations.loc[:,['id','lat','long']]
innerJoin = pd.merge(googlePlotMap,stationPlot, on='id', how='inner')


gmap = gmplot.GoogleMapPlotter(37.336, -121.894074, 16)
gmap.heatmap(innerJoin.lat, innerJoin.long)  
gmap.draw("startStationIdHeatMap.html")'''

Se puede ver que en la ciudad de San francisco se ve una mayor concentración de donde salen los viajes. A diferencia de san josé,o mountain view . Esto se debe a que en san francisco se realizan mayor cantidad de viajes.

# Análogo al plot anterior, pero a donde se dirigen los viajes.

In [ ]:
#Heatmap a donde llegan la mayor cantidad de viajes. 
'''googlePlotMap= trips_in_minute.loc[:,['start_station_id','end_station_id']]
googlePlotMap.columns = ['start_station_id' , 'id']
stationPlot = stations.loc[:,['id','lat','long']]
innerJoin = pd.merge(googlePlotMap,stationPlot, on='id', how='inner')

gmap = gmplot.GoogleMapPlotter(37.336, -121.894074, 16)
gmap.heatmap(innerJoin.lat, innerJoin.long)  
gmap.draw("endStationIdHeatMap.html")'''

Se puede ver en "endStationIdHeatMap" que en san francisco hay 2 puntos donde más llegan los viajes. En la estaciónd de trenes calltrain (y sus estaciones más cercanas) y en el puerto del ferry.

# Posiciones de las estaciones en google maps

In [ ]:
'''googlePlotMap= trips_in_minute.loc[:,['start_station_id','end_station_id']]
googlePlotMap.columns = ['id' , 'end_station_id']


groupedby = googlePlotMap.groupby('id',as_index=False).count()
stationPlot = stations.loc[:,['id','lat','long']]
innerJoin = pd.merge(groupedby,stationPlot, on='id', how='inner')

import gmplot

gmap = gmplot.GoogleMapPlotter(37.336, -121.894074, 16)
#gmap.plot(innerJoin.lat, innerJoin.long, 'cornflowerblue', edge_width=4)
for index, row in innerJoin.iterrows():
    gmap.scatter([row['lat']], [row['long']], '#FFFFFF', row['end_station_id']*0.005, marker=False)
    gmap.marker(row['lat'],row['long'],color='#FF0000',title=str(row['id']))
    
gmap.draw("mymap.html")'''

In [ ]:
'''googlePlotMap= trips_in_minute.loc[:,['start_station_id','end_station_id']]
googlePlotMap.columns = ['id' , 'end_station_id']

groupedby = googlePlotMap.groupby('id',as_index=False).count()
stationPlot = stations.loc[:,['id','lat','long']]
#stationPlot = stationPlot[stationPlot.lat > 38]
#stationPlot = stationPlot[stationPlot.long < -121]

innerJoin = pd.merge(groupedby,stationPlot, on='id', how='inner')
innerJoin =  innerJoin[innerJoin.id > 38] 
innerJoin =  innerJoin[innerJoin.id < 80] 

import gmplot
gmap.from_geocode("San Francisco")
# gmap = gmplot.GoogleMapPlotter(3.336, -121.894074, 16)
gmap.heatmap(innerJoin.lat, innerJoin.long)  

# #gmap.plot(innerJoin.lat, innerJoin.long, 'cornflowerblue', edge_width=4)
# for index, row in innerJoin.iterrows():
#     gmap.scatter([row['lat']], [row['long']], '#FFFFFF', row['end_station_id']*0.005, marker=False)
#     gmap.marker(row['lat'],row['long'],color='#FF0000',title=str(row['id']))
    
gmap.draw("mymap.html")'''

In [ ]:
from_to = trips_in_minute[trips_in_minute['end_station_id'] == 2].groupby('start_station_id').size()

# Analizamos las realaciones que hay entre las estaciones ( cantidad de viajes entre ellas) 

In [ ]:
fig, ax = plt.subplots(figsize=(18,12))        

sns.heatmap(pd.crosstab(trips_in_minute['start_station_id'],trips_in_minute['end_station_id']));

# Analizamos la disponibilidad de las bicicletas segun dia y horario. (status.csv)

In [ ]:
'''status = pd.read_csv("../input/status.csv")'''

In [ ]:
# Convierto el campo time en datetime
'''status_convert = status
status_convert.time = pd.to_datetime(status_convert.time) 

print status_convert'''

In [ ]:
'''status_convert.info(null_counts=True)'''

In [ ]:
'''status_convert.describe()'''

In [ ]:
# Agrego el campo weekday y hour, y elimino time para liberar un poco la memoria
'''status_convert['weekday'] =  pd.to_datetime(status_convert['time']).dt.weekday
status_convert['hour'] = pd.to_datetime(status_convert['time']).dt.hour
status_convert = status_convert.drop('time', 1)'''

In [ ]:
# Esto va a servir mas adelante para graficar
'''days = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado','Domingo']
labels = []
for day in days:
    for h in range(12):
        labels.append(day + ' - ' + str(h*2))'''

Segun el analisis exploratorio que se realizo anteriormente, elijo 2 de las estaciones mas usadas, 2 del medio y 2 de las menos usadas para graficar como varia la disponibilidad de bicicletas durante los dias de la semana.

Mas usadas: 70, 69
En el medio: 28, 46
Menos usadas: 21, 24
(los números son ids)

In [ ]:
# Estacion 70
'''df_station = status_convert[status_convert.station_id == 70]
print (df_station)'''
#dfGrouped = df.groupby(['weekday',df.hour-(df.hour%2)])['bikes_available'].count()
#print dfGrouped

In [ ]:
'''dfGrouped = df_station.groupby(['weekday',df_station.hour-(df_station.hour%2)])['bikes_available'].mean()
print dfGrouped'''

In [ ]:
'''plt.figure()
dfGrouped.plot(kind='bar',figsize=(30,8), title="Disponibilidad de bicicletas segun dia y hora en la estacion San Francisco Caltrain (Townsend at 4th)").set_xticklabels(labels)
plt.xlabel("dia - hora")'''
#plt.ylabel("disponibilidad de bicicletas")

In [ ]:
# Estación 69
'''df_station = status_convert[status_convert.station_id == 69]
dfGrouped = df_station.groupby(['weekday',df_station.hour-(df_station.hour%2)])['bikes_available'].mean()
plt.figure()
dfGrouped.plot(kind='bar',figsize=(30,8), title="Disponibilidad de bicicletas segun dia y hora San Francisco Caltrain 2 (330 Townsend)").set_xticklabels(labels)
plt.xlabel("dia - hora")'''
#plt.ylabel("disponibilidad de bicicletas")

In [ ]:
# Estación 28
'''df_station = status_convert[status_convert.station_id == 28]
dfGrouped = df_station.groupby(['weekday',df_station.hour-(df_station.hour%2)])['bikes_available'].mean()
plt.figure()
dfGrouped.plot(kind='bar',figsize=(30,8), title="Disponibilidad de bicicletas segun dia y hora Mountain View Caltrain Station").set_xticklabels(labels)
plt.xlabel("dia - hora")'''
#plt.ylabel("disponibilidad de bicicletas")

In [ ]:
# Estación 46
'''df_station = status_convert[status_convert.station_id == 46]
dfGrouped = df_station.groupby(['weekday',df_station.hour-(df_station.hour%2)])['bikes_available'].mean()
plt.figure()
dfGrouped.plot(kind='bar',figsize=(30,8), title="Disponibilidad de bicicletas segun dia y hora Washington at Kearney").set_xticklabels(labels)
plt.xlabel("dia - hora")'''
#plt.ylabel("disponibilidad de bicicletas")

In [ ]:
# Estación 21
'''df_station = status_convert[status_convert.station_id == 21]
dfGrouped = df_station.groupby(['weekday',df_station.hour-(df_station.hour%2)])['bikes_available'].mean()
plt.figure()
dfGrouped.plot(kind='bar',figsize=(30,8), title="Disponibilidad de bicicletas segun dia y hora Franklin at Maple").set_xticklabels(labels)
plt.xlabel("dia - hora")'''
#plt.ylabel("disponibilidad de bicicletas")

In [ ]:
# Estación 24
'''df_station = status_convert[status_convert.station_id == 24]
dfGrouped = df_station.groupby(['weekday',df_station.hour-(df_station.hour%2)])['bikes_available'].mean()
plt.figure()
dfGrouped.plot(kind='bar',figsize=(30,8), title="Disponibilidad de bicicletas segun dia y hora Redwood City Public Library").set_xticklabels(labels)
plt.xlabel("dia - hora")'''
#plt.ylabel("disponibilidad de bicicletas")

Según se pudo observar en los gráficos, las diferencias más notorias se dan en las estaciones que tienen más tráfico.
En las estaciones más usadas se nota una clara diferencia en la disponibilidad de las bicicletas entre las 8 y las 16 horas de los días hábiles, habiendo menos disponibilidad. En las estaciones del medio la diferencia se sigue notando, pero ya no es tan marcada, mientras que en las estaciones menos usadas ya no se alcanza a notar un patron claro.


# Analizamos la cantidad de viajes por bicicleta

In [ ]:
bikes = trips_in_minute.loc[:,'bike_id']
bikes

In [ ]:
print ((trips_in_minute.bike_id.unique()).min())
print ((trips_in_minute.bike_id.unique()).max()) 
print (len(trips_in_minute.bike_id.unique())) 

Hay 700 bicicletas, donde el bike_id esta entre 9 y 878

In [ ]:
#%matplotlib notebook
bikes.plot.hist(alpha=0.5, title="Cantidad de viajes por bicicleta")
#plt.xlabel("bike_id")
#plt.ylabel("cantidad de viajes")

Se puede observar que las bicicletas mas usadas (con mayor cantidad de viajes) estan en el rango 300 y 600 (bike_id)

# Analizamos la relacion entre bicicletas y estaciones

In [ ]:

trips_in_minute.plot.scatter('start_station_id','bike_id',alpha=1,title = "Relacion bicicletas-estaciones de salida",figsize=(15,8));

In [ ]:

trips_in_minute.plot.scatter('end_station_id','bike_id',alpha=1,title = "Relacion bicicletas-estaciones de llegada",figsize=(15,8));

El patron observado es aproximado para las estaciones de salida y llegada.
Se puede apreciar que el rango de bicicletas mas usadas (300-600) son utilizadas, en mayor cantidad de viajes, por las estaciones 69 y 70 (estaciones mas usadas) y las bicicletas menos usadas son utilizadas, en mayor cantidad de viajes, por las estaciones 21 y 24 (estaciones menos usadas).